In [ ]:
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.3 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y pmdarima numpy
!pip install pmdarima numpy

Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
  Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
#import pmdarima as pm
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('Revenue.xlsx - Data.csv')

In [ ]:
data['Rev'] = pd.to_numeric(data['Rev'].str.replace(',', '', regex=True))
data.head()

,Client Key,Type,Treatment Type,DOS,Rev
0,AC-000836,Product 1,Initial Consult & Ancillaries,1/1/2016,2033.27
1,AC-000836,Product 1,IUI,1/1/2016,1563.56
2,AC-000836,Product 1,IVF Freeze-All,1/1/2016,11817.29
3,AC-000836,Product 1,Storage,1/1/2016,90.00
4,AC-000836,Product 1,Traditional Fresh IVF,1/1/2016,2236.34


In [ ]:
len(data)

22199

## Questions About The Data:

*   Are there missing values in the revenue data?
*   How many unique treatments are within each product type?
*   What is the total timeframe for our data?
*   How similar are the revenues for different instances of the same treatment? Does the distribution of revenue from a specific treatment significantly change over time?


In [ ]:
missing_data = data.isnull().sum()
print(missing_data)

This tells us that there are 46 clients that have Product 1 but not Product 2. Now lets try to understand more about the treatments falling under each product.

In [ ]:
product_1_treatments = data[data['Type'] == 'Product 1']['Treatment Type'].unique()
product_2_treatments = data[data['Type'] == 'Product 2']['Treatment Type'].unique()

In [ ]:
print(product_1_treatments)
print(product_2_treatments)

In [ ]:
elements_not_in_array2 = np.setdiff1d(product_1_treatments, product_2_treatments)
print(elements_not_in_array2)

In [ ]:
elements_not_in_array1 = np.setdiff1d(product_2_treatments, product_1_treatments)
print(elements_not_in_array1)

We find that the only difference between products is that Product 2 does not include a Storage treatment like Product 1 does.

For the following two blocks, I was curious as to the timeframe of our data and what each individual timestep was. I found the timeframe to be January 2016 to April 2021 and each datapoint represents a revenue value for the month that it happened in. Thus, our timestep is monthly.

In [ ]:
data['DOS'] = pd.to_datetime(data['DOS'])
start_date = data['DOS'].min()
end_date = data['DOS'].max()
print(start_date, end_date)

In [ ]:
unique_dos_by_product = data.groupby('Type')['DOS'].unique()

for product_type, dos_list in unique_dos_by_product.items():
  print(f"Product Type: {product_type}")
  for dos in dos_list:
    print(dos)
  print("-" * 20)


# Sample SQL Queries

In [ ]:
conn = sqlite3.connect(":memory:")
data.to_sql("data", conn, index=False, if_exists="replace")

### Annual Revenue Growth by Year for Both Products

In [ ]:
query = """
WITH YearlyRevenue AS (
    SELECT
        strftime('%Y', DOS) AS Year,
        Type,
        SUM(Rev) AS TotalRevenue
    FROM data
    GROUP BY Year, Type
),
Product1RevenueGrowth AS (
    SELECT
        Year,
        TotalRevenue,
        TotalRevenue - LAG(TotalRevenue, 1, 0) OVER (ORDER BY Year) AS `Product 1 Revenue Growth`
    FROM YearlyRevenue
    WHERE Type = 'Product 1'
)
SELECT * FROM Product1RevenueGrowth;
"""

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
query = """
WITH YearlyRevenue AS (
    SELECT
        strftime('%Y', DOS) AS Year,
        Type,
        SUM(Rev) AS TotalRevenue
    FROM data
    GROUP BY Year, Type
),
Product2RevenueGrowth AS (
    SELECT
        Year,
        TotalRevenue,
        TotalRevenue - LAG(TotalRevenue, 1, 0) OVER (ORDER BY Year) AS `Product 2 Revenue Growth`
    FROM YearlyRevenue
    WHERE Type = 'Product 2'
)
SELECT * FROM Product2RevenueGrowth;
"""

df_result = pd.read_sql_query(query, conn)
df_result

##### *Important to note that our data ends in April 2021 so the revenue growth for 2021 is inaccurate as the full years worth of data is not  available.

### New Sales vs Returning Revenue Breakdown

In [ ]:
query = """
WITH FirstPurchase AS (
    SELECT "Client Key", MIN(DOS) AS first_purchase_date
    FROM data
    GROUP BY "Client Key"
)
SELECT
    strftime('%Y', d.DOS) AS year,
    SUM(CASE WHEN d.DOS = fp.first_purchase_date AND d.Type = 'Product 1' THEN d.Rev ELSE 0 END) AS product_1_new_sales_revenue,
    SUM(CASE WHEN d.DOS = fp.first_purchase_date AND d.Type = 'Product 2' THEN d.Rev ELSE 0 END) AS product_2_new_sales_revenue,
    SUM(CASE WHEN d.DOS > fp.first_purchase_date AND d.Type = 'Product 1' THEN d.Rev ELSE 0 END) AS product_1_returning_revenue,
    SUM(CASE WHEN d.DOS > fp.first_purchase_date AND d.Type = 'Product 2' THEN d.Rev ELSE 0 END) AS product_2_returning_revenue
FROM data d
LEFT JOIN FirstPurchase fp ON d."Client Key" = fp."Client Key"
GROUP BY year
ORDER BY year;
"""

df_result = pd.read_sql_query(query, conn)
df_result

## Analysis of Findings from SQL Queries


*   Annual revenue was increasing fast before 2020. This is almost certainly due to the COVID-19 pandemic's effects on the economy
*   A significant majority of the revenue is returning. Indicating that the service is solid enough for customers to continue buying the products.





# Forecasting Revenue for both products until the end of 2021.

Plot cumulative revenue - it appears to grow exponentially.

I wanted to look at this to get a sense of the trend in the time series. Take a look at the hitch around March 2020.

In [ ]:
revenue_by_date = data.groupby('DOS')['Rev'].sum()
cumulative_revenue = revenue_by_date.cumsum()

plt.figure(figsize=(10, 6))
plt.plot(cumulative_revenue.index, cumulative_revenue.values)
plt.xlabel('Date')
plt.ylabel('Cumulative Revenue')
plt.title('Cumulative Revenue over Time')
plt.grid(True)
plt.show()


Now let's look at the monthly revenue for both products combined. We can see that there is a solid trend upward with some seasonality as well. Subtle increases at the beginning of the years until around December there is negative seasonality until revenue increases again in Janaury, usually offsetting the late year decline. This is likely due to the products being health plans and customers buying their yearly plan in January. There is likely an incentive to this as we see less revenue consistently toward the end of the year.

In [ ]:
monthly_revenue = data.groupby('DOS')['Rev'].sum()

plt.figure(figsize=(10, 6))
plt.plot(monthly_revenue.index, monthly_revenue.values)
plt.xlabel('Date')
plt.ylabel('Monthly Revenue')
plt.title('Monthly Revenue over Time')
plt.grid(True)
plt.show()

Now we will look at the products separately. We see a lot of the same trends that we identified above in the individual time series but at different scales. This is because Product 2 appears to have been launched 2 years exactly after Product 1 and they both have almost the same components.

In [ ]:
monthly_revenue_by_product = data.groupby(['DOS', 'Type'])['Rev'].sum().reset_index()
monthly_revenue_product_1 = monthly_revenue_by_product[monthly_revenue_by_product['Type'] == 'Product 1']
monthly_revenue_product_2 = monthly_revenue_by_product[monthly_revenue_by_product['Type'] == 'Product 2']

plt.figure(figsize=(10, 6))
plt.plot(monthly_revenue_product_1['DOS'], monthly_revenue_product_1['Rev'])
plt.plot(monthly_revenue_product_2['DOS'], monthly_revenue_product_2['Rev'])
plt.xlabel('Date')
plt.ylabel('Monthly Revenue by Product')
plt.title('Monthly Revenue by Product over Time')
plt.legend(['Product 1', 'Product 2'])
plt.grid(True)
plt.show()

I decided to apply a log-transform to monthly revenue in order to offset some of the sharp upward trend. This may also help with stationarity when we are looking to fit a model.

In [ ]:
monthly_revenue_by_product = data.groupby(['DOS', 'Type'])['Rev'].sum().reset_index()
monthly_revenue_product_1 = monthly_revenue_by_product[monthly_revenue_by_product['Type'] == 'Product 1']
monthly_revenue_product_2 = monthly_revenue_by_product[monthly_revenue_by_product['Type'] == 'Product 2']

plt.figure(figsize=(10, 6))
plt.plot(monthly_revenue_product_1['DOS'], np.log(monthly_revenue_product_1['Rev']))
plt.plot(monthly_revenue_product_2['DOS'], np.log(monthly_revenue_product_2['Rev']))
plt.xlabel('Date')
plt.ylabel('Log Monthly Revenue by Product')
plt.title('Log Monthly Revenue by Product over Time')
plt.grid(True)
plt.show()

Now we will begin to test for stationarity with the Augmented Dickey-Fuller (ADF) Test. Stationarity is the main assumption of ARIMA family models like the one we will build and without it our forecasts will be less reliable.

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(monthly_revenue_product_1['Rev'])
adf_stat, p_value, usedlag, nobs, critical_values, icbest = result

print(f'ADF Statistic: {adf_stat}')
print(f'p-value: {p_value}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}, {value}')

This result from ADF indicates that the raw time series for Product 1 is not stationary with p-value = 0.99.

Let's apply the log-transform and see if we achieve stationarity with that.

In [ ]:
monthly_revenue_product_1['Log_Rev'] = np.log(monthly_revenue_product_1['Rev'])
monthly_revenue_product_2['Log_Rev'] = np.log(monthly_revenue_product_2['Rev'])

result = adfuller(monthly_revenue_product_1['Log_Rev'])
adf_stat, p_value, usedlag, nobs, critical_values, icbest = result

print(f'ADF Statistic: {adf_stat}')
print(f'p-value: {p_value}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}, {value}')

The time series is still not stationary with p-value = 0.7.

We can also apply differencing to try and achieve stationarity.

In [ ]:
monthly_revenue_product_1['Log_Rev_Diff'] = monthly_revenue_product_1['Log_Rev'].diff()
monthly_revenue_product_2['Log_Rev_Diff'] = monthly_revenue_product_2['Log_Rev'].diff()

monthly_revenue_product_1 = monthly_revenue_product_1.dropna()
monthly_revenue_product_2 = monthly_revenue_product_2.dropna()

result = adfuller(monthly_revenue_product_1['Log_Rev_Diff'])
adf_stat, p_value, usedlag, nobs, critical_values, icbest = result

print(f'ADF Statistic: {adf_stat}')
print(f'p-value: {p_value}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}, {value}')

In [ ]:
result = adfuller(monthly_revenue_product_2['Log_Rev_Diff'], autolag='AIC')
adf_stat, p_value, usedlag, nobs, critical_values, icbest = result

print(f'ADF Statistic: {adf_stat}')
print(f'p-value: {p_value}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}, {value}')

This result from ADF test indicates that the differenced log-transform of the raw time series is indeed stationary. We can now proceed with our SARIMAX modeling.

I wanted to use an exogenous economic indicator in the model so I downloaded data from FRED on GDP. This GDP data is quarterly so I had to forward fill values in order to align with the monthly nature of our time series. Hopefully the inclusion of GDP will soften the impact of the COVID-19 pandemic on our data as GDP fell during this time too.

In [ ]:
import pandas_datareader.data as web
gdp_p1 = web.DataReader('GDP', 'fred', "2016-01-01", "2021-04-01")
gdp_p1_monthly = gdp_p1.resample('MS').last()
gdp_p1_monthly['GDP'] = gdp_p1_monthly['GDP'].ffill()
gdp_p1_monthly.plot()

gdp_p2 = web.DataReader('GDP', 'fred', "2018-01-01", "2021-04-01")
gdp_p2_monthly = gdp_p2.resample('MS').last()
gdp_p2_monthly['GDP'] = gdp_p2_monthly['GDP'].ffill()
gdp_p2_monthly.plot()
plt.show()

In [ ]:
gdp_p1_monthly = gdp_p1_monthly[:-1]
gdp_p2_monthly = gdp_p2_monthly[:-1]

gdp_p1_monthly.index = monthly_revenue_product_1['DOS']
gdp_p2_monthly.index = monthly_revenue_product_2['DOS']

monthly_revenue_product_1['GDP'] = gdp_p1_monthly.values
monthly_revenue_product_2['GDP'] = gdp_p2_monthly.values

I am also going to include a COVID-19 indicator that is equal to zero at every timestep except for those that are outliers due to COVID-19. This will hopefully tell the model that these are not trends that we would expect to reoccur.

In [ ]:
monthly_revenue_product_1['COVID'] = 0
monthly_revenue_product_2['COVID'] = 0

covid_start = '2020-03-01'
covid_end = '2020-06-01'

monthly_revenue_product_1.loc[(monthly_revenue_product_1['DOS'] >= covid_start) & (monthly_revenue_product_1['DOS'] <= covid_end), 'COVID'] = 1
monthly_revenue_product_2.loc[(monthly_revenue_product_2['DOS'] >= covid_start) & (monthly_revenue_product_2['DOS'] <= covid_end), 'COVID'] = 1


Now for model building. We will fit a SARIMAX model (Seasonal Autoregressive Integrated Moving Average) to try and model revenue. In order to tune the hyperparameters we will use an auto ARIMA search which tests different sets of hyperparameters until it finds the model with the lowest AIC. AIC is a metric that tells you how well the model fits the data it was trained on while penalizing models with more parameters. This is a great metric since other goodness of fit metrics like R-squared can be inflated simply by making your model more complex. This is known as overfitting when your model isn't actually modeling the underlying relatonship and instead is modeling the random noise in your data.

Once the best parameters are found, we will calculate our forecasts on the out of sample test data to evaluate performance. I am using Mean Absolute Percentage Error (MAPE) for this purpose. MAPE tells us how much our forecasts were off by as a percentage, on average. I like MAPE in this context because it is very intuitive and we are predicting large numbers so Mean Squared Error may be a bit confusing as it will also be a very large number.

One quick note to avoid confusion: I modeled with log revenue instead of the differenced log revenue whiich we found to be stationary. The reason for this is that the SARIMAX model differences the data for us by setting d=1, indicating a first differencing of our data. Auto ARIMA found this to be optimal for both models as well.

In [ ]:
monthly_revenue_product_1['Log_Rev'] = np.log(monthly_revenue_product_1['Rev'])
monthly_revenue_product_2['Log_Rev'] = np.log(monthly_revenue_product_2['Rev'])

train_p1 = monthly_revenue_product_1[:-8]['Log_Rev']
test_p1 = monthly_revenue_product_1[-8:]['Log_Rev']
train_p2 = monthly_revenue_product_2[:-8]['Log_Rev']
test_p2 = monthly_revenue_product_2[-8:]['Log_Rev']

exog_train_p1 = monthly_revenue_product_1[:-8][['GDP', 'COVID']]
exog_test_p1 = monthly_revenue_product_1[-8:][['GDP', 'COVID']]
exog_train_p2 = monthly_revenue_product_2[:-8][['GDP', 'COVID']]
exog_test_p2 = monthly_revenue_product_2[-8:][['GDP', 'COVID']]

exog_train_p1.index = train_p1.index
exog_train_p2.index = train_p2.index
exog_test_p1.index = test_p1.index
exog_test_p2.index = test_p2.index

auto_p1 = pm.auto_arima(train_p1, exogenous= exog_train_p1, seasonal=True, m=12, stepwise=True, suppress_warnings=True)
auto_p2 = pm.auto_arima(train_p2, exogenous= exog_train_p2, seasonal=True, m=12, stepwise=True, suppress_warnings=True)

print(f"\nBest SARIMA Order for Product 1: {auto_p1.order}, Seasonal: {auto_p1.seasonal_order}")
print(f"Best SARIMA Order for Product 2: {auto_p2.order}, Seasonal: {auto_p2.seasonal_order}")

model_p1 = SARIMAX(train_p1, order=auto_p1.order, seasonal_order=auto_p1.seasonal_order, exog=exog_train_p1, enforce_stationarity=True, enforce_invertibility=True)
model_p2 = SARIMAX(train_p2, order=auto_p2.order, seasonal_order=auto_p2.seasonal_order, exog=exog_train_p2, enforce_stationarity=True, enforce_invertibility=True)

results_p1 = model_p1.fit()
results_p2 = model_p2.fit()

forecast_p1 = results_p1.get_forecast(steps=len(test_p1), exog=exog_test_p1)
forecast_p2 = results_p2.get_forecast(steps=len(test_p2), exog=exog_test_p2)

preds_p1 = np.exp(forecast_p1.predicted_mean)
ci_p1 = np.exp(forecast_p1.conf_int())
preds_p2 = np.exp(forecast_p2.predicted_mean)
ci_p2 = np.exp(forecast_p2.conf_int())

mape_p1 = mean_absolute_percentage_error(np.exp(test_p1), preds_p1)
mape_p2 = mean_absolute_percentage_error(np.exp(test_p2), preds_p2)
print(f"\nMAPE Product 1: {mape_p1:.2f}, MAPE Product 2: {mape_p2:.2f}")


residuals_p1 = results_p1.resid
residuals_p2 = results_p2.resid

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(residuals_p1, bins=200, alpha=0.7, color='blue')
plt.xlim([-0.5, 0.5])
plt.ylim([0, 20])
plt.title("Residuals Histogram - Product 1")

plt.subplot(1,2,2)
plt.hist(residuals_p2, bins=80, alpha=0.7, color='green')
plt.xlim([-0.5, 0.5])
plt.ylim([0, 20])
plt.title("Residuals Histogram - Product 2")

plt.show()


Once our model is fit and predictions are made, we can analyze the residuals. They appear to be fairly normally distributed around zero with a few outliers that are not shown here as I have zoomed in on where most of the data lies.

Since we are using GDP as an exogenous variable, we must estimate it for the forecasting range as well.

In [ ]:
train_gdp_p1 = monthly_revenue_product_1['GDP']
auto_gdp_p1 = pm.auto_arima(train_gdp_p1, seasonal=True, m=12, stepwise=True, suppress_warnings=True)
model_gdp_p1 = SARIMAX(train_gdp_p1, order=auto_gdp_p1.order, seasonal_order=auto_gdp_p1.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
results_gdp_p1 = model_gdp_p1.fit()

forecast_gdp_p1 = results_gdp_p1.get_forecast(steps=len(test_p1))
preds_gdp_p1 = np.exp(forecast_gdp_p1.predicted_mean)

train_gdp_p2 = monthly_revenue_product_2['GDP']
auto_gdp_p2 = pm.auto_arima(train_gdp_p2, seasonal=True, m=12, stepwise=True, suppress_warnings=True)
model_gdp_p2 = SARIMAX(train_gdp_p2, order=auto_gdp_p2.order, seasonal_order=auto_gdp_p2.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
results_gdp_p2 = model_gdp_p2.fit()

forecast_gdp_p2 = results_gdp_p2.get_forecast(steps=len(test_p2))
preds_gdp_p2 = np.exp(forecast_gdp_p2.predicted_mean)

In [ ]:
forecast_gdp_new_dates = pd.date_range(start='2021-05-01', end='2021-12-01', freq='MS')

new_forecast_gdp_p1 = results_gdp_p1.get_forecast(steps=len(forecast_gdp_new_dates))
gdp_p1_forecasts = new_forecast_gdp_p1.predicted_mean
gdp_p1_forecasts = pd.Series(gdp_p1_forecasts.values, index=forecast_gdp_new_dates, name = 'GDP')

new_forecast_gdp_p2 = results_gdp_p2.get_forecast(steps=len(forecast_gdp_new_dates))
gdp_p2_forecasts = new_forecast_gdp_p2.predicted_mean
gdp_p2_forecasts = pd.Series(gdp_p2_forecasts.values, index=forecast_gdp_new_dates, name = 'GDP')

In [ ]:
forecast_p1_test_dates = pd.date_range(start='2020-09-01', end='2021-04-01', freq='MS')
forecast_p2_test_dates = pd.date_range(start='2020-09-01', end='2021-04-01', freq='MS')

test_forecast_p1 = pd.Series(preds_p1.values, index=forecast_p1_test_dates, name = 'Rev')
test_forecast_p2 = pd.Series(preds_p2.values, index=forecast_p2_test_dates, name = 'Rev')

In [ ]:
forecast_p1_new_dates = pd.date_range(start='2021-05-01', end='2021-12-01', freq='MS')
forecast_p2_new_dates = pd.date_range(start='2021-05-01', end='2021-12-01', freq='MS')

covid_indicator = pd.Series(0, index=gdp_p1_forecasts.index, name='COVID')
exog_p1 = pd.concat([gdp_p1_forecasts, covid_indicator], axis=1)
exog_p2 = pd.concat([gdp_p2_forecasts, covid_indicator], axis=1)
exog_p1 = exog_p1.astype(float).reset_index(drop=True)
exog_p2 = exog_p2.astype(float).reset_index(drop=True)

new_forecast_p1 = results_p1.get_forecast(steps=len(forecast_p1_new_dates), exog=exog_p1.values)
new_forecast_p2 = results_p2.get_forecast(steps=len(forecast_p2_new_dates), exog=exog_p2.values)

new_preds_p1 = np.exp(new_forecast_p1.predicted_mean)
new_ci_p1 = np.exp(new_forecast_p1.conf_int())
new_preds_p2 = np.exp(new_forecast_p2.predicted_mean)
new_ci_p2 = np.exp(new_forecast_p2.conf_int())

new_forecast_p1 = pd.Series(new_preds_p1.values, index=forecast_p1_new_dates, name = 'Rev')
new_forecast_p2 = pd.Series(new_preds_p2.values, index=forecast_p2_new_dates, name = 'Rev')

Now we will plot the actual time series, the test data predictions of our model, and the forecast for the rest of 2021.

In [ ]:
plt.plot(monthly_revenue_product_1['DOS'], monthly_revenue_product_1['Rev'], label='Actual Data')
plt.plot(forecast_p1_test_dates, test_forecast_p1, label = 'Test Forecast')
plt.plot(new_forecast_p1, label = 'Remaining Forecast')
plt.fill_between(new_forecast_p1.index, new_ci_p1['lower Log_Rev'].values, new_ci_p1['upper Log_Rev'].values, color='blue', alpha=0.15)
plt.ylim([0, 12000000])
plt.xlabel("Monthly Revenue")
plt.ylabel("Date")
plt.title("Product 1 Monthly Revenue + Remainder of 2021 Forecast")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(monthly_revenue_product_2['DOS'], monthly_revenue_product_2['Rev'], label='Actual Data')
plt.plot(forecast_p2_test_dates, test_forecast_p2, label = 'Test Forecast')
plt.plot(new_forecast_p2, label = 'Remaining Forecast')
plt.fill_between(new_forecast_p2.index, new_ci_p2['lower Log_Rev'].values, new_ci_p2['upper Log_Rev'].values, color='blue', alpha=0.15)
plt.ylim([0, 4000000])
plt.xlabel("Monthly Revenue")
plt.ylabel("Date")
plt.title("Product 2 Monthly Revenue + Remainder of 2021 Forecast")
plt.legend()
plt.grid(True)
plt.show()

We can see that the test predictions are actually alright for both models. Product 2's model I would even consider using for future forecasting as it seems like the rresults it produces are completely reasonable. However, Product 1's forecast is incredibly erratic and very uncertain. For this reason I chose to employ a stronger time series model to try and get a better forecast. SARIMAX models are fantastic in that they are very interpretable but this simplicity makes it harder to forecast with them as you have to really know a lot about your data in order to model the correct relationship.

Below are the final SARIMAX model summaries for both Products. You can see that we achieved 18% MAPE for Product 1 and 14% for Product 2 on the test data. These are both solid scores for MAPE.

In [ ]:
print(f"\nMAPE Product 1: {mape_p1:.2f}, MAPE Product 2: {mape_p2:.2f}\n")
print(results_p1.summary())
print(results_p2.summary())

Now we will model monthly revenue for both products with Meta's Prophet Time Series model. It is much more complex than SARIMAX and you don't even need your data to be stationary. That's why we are using raw revenue instead of log revenue or differenced log revenue. Powerful models like Prophet are able to identify complex relationships in your data that you may not even be able to see yourself. That is why I employ it here. We will fit the model with several sets of parameters and choose the model with the lowest MAPE so long as it comes with a reasonable forecast and reasonable uncertainty in its predictions.

In [ ]:
def forecast_with_prophet(df, product_name, changepoint_prior_scale, seasonality_prior_scale, n_changepoints, seasonality_mode):
    df = df.rename(columns={'DOS': 'ds', 'Rev': 'y'})
    train_data = df[df['ds'] < '2021-01-01']
    test_data = df[(df['ds'] >= '2021-01-01') & (df['ds'] <= '2021-04-01')]

    model = Prophet(changepoint_prior_scale=changepoint_prior_scale, seasonality_prior_scale=seasonality_prior_scale, n_changepoints=n_changepoints, seasonality_mode=seasonality_mode)
    model.fit(train_data)

    future_dates = pd.DataFrame({'ds': pd.date_range(start=train_data['ds'].min(), end='2021-12-01', freq='MS')})
    forecast = model.predict(future_dates)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(df['ds'], df['y'], label='Actual Data', color='blue')

    ax.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='red')
    ax.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='red', alpha=0.3, label='Confidence Interval')

    ax.plot(test_data['ds'], forecast['yhat'][len(train_data):len(train_data)+len(test_data)], label='Test Set Predictions', color='green', linestyle='--')

    ax.set_title(f"Prophet Forecast for {product_name}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Revenue")
    ax.legend()
    plt.show()

    forecasted_revenue_test = forecast['yhat'][len(train_data):len(train_data)+len(test_data)].values
    actual_revenue_test = test_data['y'].values
    mape = mean_absolute_percentage_error(actual_revenue_test, forecasted_revenue_test)
    mse = mean_squared_error(actual_revenue_test, forecasted_revenue_test)
    print(f"Prophet Forecast for {product_name}:")
    print(f"Test Set MAPE: {mape:.2f}")
    print(f"Test Set MSE: {mse:.2f}")

    return forecast

param_list = [
    {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10, 'n_changepoints': 25, 'seasonality_mode': 'additive'},
    {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 10, 'n_changepoints': 25, 'seasonality_mode': 'additive'},
    {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 10, 'n_changepoints': 25, 'seasonality_mode': 'additive'},
    {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10, 'n_changepoints': 10, 'seasonality_mode': 'additive'},
    {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10, 'n_changepoints': 25, 'seasonality_mode': 'multiplicative'},
    {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 15, 'n_changepoints': 25, 'seasonality_mode': 'additive'},
    {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 15, 'n_changepoints': 10, 'seasonality_mode': 'multiplicative'}
]

for params in param_list:
    print("Parameters:", params)
    forecast_with_prophet(monthly_revenue_product_1, "Product 1", params['changepoint_prior_scale'], params['seasonality_prior_scale'], params['n_changepoints'], params['seasonality_mode'])

for params in param_list:
    print("Parameters:", params)
    forecast_with_prophet(monthly_revenue_product_2, "Product 2", params['changepoint_prior_scale'], params['seasonality_prior_scale'], params['n_changepoints'], params['seasonality_mode'])


Parameters: {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10, 'n_changepoints': 25, 'seasonality_mode': 'additive'}


NameError: name 'monthly_revenue_product_1' is not defined

I have chosen the following two models for forecasting. It turns out that the hyperparameters that produced the best MAPE on test set were the same for both products.

In [ ]:
forecast_product_1 = forecast_with_prophet(monthly_revenue_product_1, "Product 1", 0.05, 10, 10, 'additive')
forecast_product_2 = forecast_with_prophet(monthly_revenue_product_2, "Product 2", 0.05, 10, 10, 'additive')

In [ ]:
forecast_product_1 = forecast_product_1.set_index('ds')
forecast_product_2 = forecast_product_2.set_index('ds')

In [ ]:
preds_p1 = forecast_product_1['yhat'].iloc[-8:].values
preds_p2 = forecast_product_2['yhat'].iloc[-8:].values

In [ ]:
log_monthly_revenue_product_1 = np.log(monthly_revenue_product_1['Rev'])
log_monthly_revenue_product_2 = np.log(monthly_revenue_product_2['Rev'])

In [ ]:
yearly_revenue_product_1 = monthly_revenue_product_1.groupby(monthly_revenue_product_1['DOS'].dt.year)['Rev'].sum()
yearly_revenue_product_2 = monthly_revenue_product_2.groupby(monthly_revenue_product_2['DOS'].dt.year)['Rev'].sum()

print("Product 1 Forecasted Total Revenue for 2021: ", np.sum(preds_p1) + yearly_revenue_product_1.values[-1])
print("Product 2 Forecasted Total Revenue for 2021: ", np.sum(preds_p2) + yearly_revenue_product_2.values[-1])

#### Cumulative Revenue with forecasts added in at the end

I like the simplicity of this plot as it tells us that our Prophet forecast is in line with the trend we see in cumulative revenue.

In [ ]:
revenue_by_date = data.groupby('DOS')['Rev'].sum()
preds_series = pd.Series(preds_p1 + preds_p2, index=pd.date_range(start='2021-05-01', end='2021-12-01', freq='MS'))
total_revenue = pd.concat([revenue_by_date, preds_series])
cumulative_revenue = total_revenue.cumsum()

plt.figure(figsize=(10, 6))
plt.plot(cumulative_revenue.index, cumulative_revenue.values)
plt.xlabel('Date')
plt.ylabel('Cumulative Revenue')
plt.title('Cumulative Revenue over Time')
plt.grid(True)
plt.show()

# Reflection on Modeling Task

When it came to the time series forecasting step I wanted to build a SARIMAX model for simplicity. I first attempted to get my data stationary with a log transform. This did not pass ther Augmented Dickey-Fuller Test for stationarity. Nest step was to take a first difference of the revenue data and this ended up being stationary for both products. I then went on to use autocorrelation and partial autocorrelation to understand more about the lags of the revenue data. I had decided that this model would be better if it included an AR(1) component and ended up removing the MA(1) component due to numerical instability of the parameter estimates due to multicollinearity. Through several tries with different hyperparameters I could not seem to find a good fit for both products. I ended up adding GDP as an exogenous predictor and it did seem to help. Another issue I encountered was the COVID-19 pandemic's impact on revenue. It seemed as if the model had learned this as a seasonal trend so I accounted for that by interpolating over it and eventually even removing those outlying months. Neither of these solutions seemed to work. My solution for the COVID outliers was to add an indicator variable for the months with outlying values. This seemed to help but I still could not get a great fit for either of the models even after I began hyperparameter stepwise searching. I concluded that the data may be too complex to model with SARIMAX. I did end up achieving 18% MAPE and 14% MAPE on Product 1 and Product 2's out of sample test data, respectively. My decision to move forward with another model was based on the forecasting I saw after that was very erratic from Product 1's model.

To overcome this complexity, I chose to forecast instead with Meta's Prophet Time Series model. This was very simple to use and ended up producing reasonable forecasts for the remainder of 2021. I researched a bit about the hyperparameters and then set up a list of different combinations to try for each of the Product's models. In the end I chose the model with the most reasonable uncertainty bounds and the lowest Mean Absolute Percent Error (MAPE) which ended up being the same set of hyperparameters for each model.

Overall I would like to spend more time analyzing the data in order to find a good fit for a more intuitive time series model like SARIMAX. I learned that the benefits of a simple model used to forecast complex data are only earned through mastery of the data you're working with. More powerful models are great and very helpful but they do not require the same skill to achieve favorable results. I still really enjoyed my attempt to model this data with SARIMAX and am honestly thrilled with the results for the SARIMAX model of Product 2.